In [ ]:
# Installations to run in Planetary Computer environment - uncomment for first time running in environment
#import sys
#!{sys.executable} -m pip install pip earthengine-api
#!{sys.executable} -m pip install pip geemap

In [ ]:
import ee
#ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import geopandas as gpd

In [ ]:
## specify areas of interest / districts and metadata
## URL method accessed an UrbanShift city's boundaries and uses information from file name and geoBoundaries properties ("geo_name") to create properties for output file
URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary-CRI-San_Jose-ADM2.geojson'
#'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/ADM1/boundary-MAR-Marrakech-ADM1.geojson'
DistrictsGJ = requests.get(URL).json()
Districts = geemap.geojson_to_ee(DistrictsGJ)

#URL = 'https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/urban_edge_t3.geojson'
#DistrictsGDF = gpd.read_file(URL)
#DistrictsGDF.sample(3)
#Districts = geemap.gdf_to_ee(DistrictsGDF)

#Districts = ee.FeatureCollection('users/emackres/Wards/Addis_Ababa_Woredas')
#Districts = ee.FeatureCollection('projects/wri-datalab/AUE/urban_edge/urban_edge_t3').first()

cityname = os.path.splitext(os.path.basename(URL))[0].split('-',2)[2].rsplit('-',1)[0]
def Rename(feat):
    return feat.set('geo_name',cityname)
#Districts = Districts.union(1).map(Rename)

DistrictsProjCRS = Districts.geometry().projection().crs()
print(DistrictsProjCRS.getInfo())
print(Districts.first().getString('geo_name').getInfo())

In [ ]:
# extract area properties from standarized filename
# https://note.nkmk.me/en/python-split-rsplit-splitlines-re/ 
basename = os.path.splitext(os.path.basename(URL))[0]
AOIname = basename.split('-',1)[1].rsplit('-',1)[0]

Areaofinterest = AOIname ## 3-letter country abreviation - city name with underscore for spaces, e.g. "ETH-Addis_Ababa"
print(Areaofinterest)

In [ ]:
## create map
Map = geemap.Map(height="350px")
Map

In [ ]:
## add basemap and center on area of interest
Map.add_basemap('HYBRID')
Map.centerObject(Districts, zoom=12)

In [ ]:
##Add Land use land cover dataset
WC = ee.ImageCollection("ESA/WorldCover/v100")
WorldCover = WC.first();

## define projection for use later
WCprojection = WC.first().projection();  
print('WorldCover projection:', WCprojection.getInfo());

Map.addLayer(WorldCover, {'bands': "Map"}, "WorldCover 10m 2020 (ESA)",1);

Map.add_legend(builtin_legend='ESA_WorldCover',position='bottomleft')

In [ ]:
## Add intra-urban land use dataset

ULU = ee.ImageCollection("projects/wri-datalab/urban_land_use/v1")

WRIulu = ULU.select('lulc').reduce(ee.Reducer.firstNonNull()).rename('lulc')
WRIulu = WRIulu.mask(WRIulu.mask().gt(0))
WRIroad = ULU.select('road_lulc').reduce(ee.Reducer.firstNonNull()).rename('lulc')
WRIuluwRoad = WRIulu.add(WRIroad).where(WRIroad.eq(1),6).mask(WRIulu.mask().gt(0))

ULUmaskedESA = WRIuluwRoad.updateMask(WorldCover.eq(50)) #.Or(WorldCover.eq(60)))

ULUmaskedESA = ULUmaskedESA.reproject(
      crs= WCprojection
    )

CLASSES_7=[
  "open_space",
  "nonresidential",
  "atomistic",
  "informal_subdivision",
  "formal_subdivision",
  "housing_project",
  "road"]
COLORS_7=[
  '33A02C',
  'E31A1C',
  'FB9A99',
  'FFFF99',
  '1F78B4',
  'A6CEE3',
  '3f3f3f']  
ULU7Params = {"bands": ['lulc'], 'min': 0, 'max': 6, "opacity": 1, "palette": COLORS_7}

#Map.addLayer(ULUmaskedESA,ULU7Params,"Urban Land Use 2020 (WRI) masked to WorldCover built",True)

In [ ]:
# set geometries and date range of interest for land surface temperature (LST) calculation

roi = Districts
ROIcenter = roi.geometry().centroid(1)

start_date = '2015-01-01'
end_date = '2022-06-28'


In [ ]:

#  CALCULATE DATES OF HOTTEST PERIOD OF HIGH TEMPERATURES FOR EACH PIXEL

# select dataset, filter by dates and visualize
dataset = (ee.ImageCollection('NASA/NEX-GDDP')
           .filter(ee.Filter.And(
               ee.Filter.date(start_date, end_date),
               ee.Filter.eq('scenario','rcp85'),
                ee.Filter.eq('model','BNU-ESM'),
               ee.Filter.bounds(Districts)
           ))
          )
AirTemperature = dataset.select(['tasmax'])
AirTemperatureVis = {
  'min': 240.0,
  'max': 300.0,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
}

#Map.addLayer(AirTemperature, AirTemperatureVis, 'Max Air Temperature')
#print(AirTemperature)

# add date as a band to image collection
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

withdates = AirTemperature.map(addDate)
#print(withdates)

# create a composite with the hottest day value and dates for every location and add to map
hottest = withdates.qualityMosaic('tasmax')
#print(hottest)
#Map.addLayer(hottest.select('tasmax'), AirTemperatureVis, 'Max temp',0)

# reduce composite to get the hottest date for centroid of ROI
resolution = dataset.first().projection().nominalScale()
NEXtempMax = ee.Number(hottest.reduceRegion(ee.Reducer.firstNonNull(), ROIcenter, resolution).get('date'))
#print(NEXtempMax.getInfo())

# convert date number to date type
date = ee.Date.parse('YYYYMMdd',str(NEXtempMax.getInfo()))
#print(date.getInfo())

# calculate 45 days before and after hottest date.  Format as short date.
start90days = date.advance(-44, 'day').format('YYYY-MM-dd')
end90days = date.advance(45, 'day').format('YYYY-MM-dd')
print(start90days.getInfo())
print(end90days.getInfo())

In [ ]:
# select parameters: date range, and landsat satellite

landsat = 'L8' # options: 'L4', 'L5', 'L7', 'L8'
date_start = start90days # or custom date in format '2020-12-20'
date_end = end90days # or custom date in format '2020-12-20'
image_limit = 100
month_start = 1
month_end = 12
use_ndvi = False

In [ ]:
#  CALCULATE MEAN LST MOSAIC FOR HOTTEST PERIOD USING LANDSAT

""""
Derived from
LSTfun = require('users/sofiaermida/landsat_smw_lst:modules/SMWalgorithm.js')
'Author': Sofia Ermida (sofia.ermida@ipma.pt; @ermida_sofia)

This code is free and open.
By using this code and any data derived with it,
you agree to cite the following reference
'in any publications derived from them':
Ermida, S.L., Soares, P., Mantas, V., Göttsche, F.-M., Trigo, I.F., 2020.
    Google Earth Engine open-source code for Land Surface Temperature estimation from the Landsat series.
    'Remote Sensing, 12 (9), 1471; https':#doi.Org/10.3390/rs12091471
"""



#LandsatLST = require('users/emackres/DataPortal:/Landsat_LST.js')
#cloudmask = require('users/emackres/DataPortal:/cloudmask.js')

COLLECTION = ee.Dictionary({
  'L4': {
    'TOA': ee.ImageCollection('LANDSAT/LT04/C01/T1_TOA'),
    'SR': ee.ImageCollection('LANDSAT/LT04/C01/T1_SR'),
    'TIR': ['B6',]
  },
  'L5': {
    'TOA': ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA'),
    'SR': ee.ImageCollection('LANDSAT/LT05/C01/T1_SR'),
    'TIR': ['B6',]
  },
  'L7': {
    'TOA': ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA'),
    'SR': ee.ImageCollection('LANDSAT/LE07/C01/T1_SR'),
    'TIR': ['B6_VCID_1','B6_VCID_2'],
  },
  'L8': {
    'TOA': ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA'),
    'SR': ee.ImageCollection('LANDSAT/LC08/C01/T1_SR'),
    'TIR': ['B10','B11']
  }
})

def NDVIaddBand(landsat):
  def wrap(image):

    # choose bands
    nir = ee.String(ee.Algorithms.If(landsat == 'L8','B5','B4'))
    red = ee.String(ee.Algorithms.If(landsat == 'L8','B4','B3'))

    # compute NDVI
    return image.addBands(image.expression('(nir-red)/(nir+red)',{
      'nir':image.select(nir).multiply(0.0001),
      'red':image.select(red).multiply(0.0001)
    }).rename('NDVI'))

  return wrap


def FVCaddBand(landsat):
  def wrap(image):

    ndvi = image.select('NDVI')

    # Compute FVC
    fvc = image.expression('((ndvi-ndvi_bg)/(ndvi_vg - ndvi_bg))**2',
      {'ndvi':ndvi,'ndvi_bg':0.2,'ndvi_vg':0.86})
    fvc = fvc.where(fvc.lt(0.0),0.0)
    fvc = fvc.where(fvc.gt(1.0),1.0)

    return image.addBands(fvc.rename('FVC'))

  return wrap


def NCEP_TPWaddBand(image):

  # first select the day of interest
  date = ee.Date(image.get('system:time_start'))
  year = ee.Number.parse(date.format('yyyy'))
  month = ee.Number.parse(date.format('MM'))
  day = ee.Number.parse(date.format('dd'))
  date1 = ee.Date.fromYMD(year,month,day)
  date2 = date1.advance(1,'days')

  # function compute the time difference from landsat image
  def datedist(image):
    return image.set('DateDist',
      ee.Number(image.get('system:time_start')) \
      .subtract(date.millis()).abs())
  

  # load atmospheric data collection
  TPWcollection = ee.ImageCollection('NCEP_RE/surface_wv') \
                  .filter(ee.Filter.date(date1.format('yyyy-MM-dd'), date2.format('yyyy-MM-dd'))) \
                  .map(datedist)

  # select the two closest model times
  closest = (TPWcollection.sort('DateDist')).toList(2)

  # check if there is atmospheric data in the wanted day
  # if not creates a TPW image with non-realistic values
  # these are then masked in the SMWalgorithm function (prevents errors)
  tpw1 = ee.Image(ee.Algorithms.If(closest.size().eq(0), ee.Image.constant(-999.0),
                      ee.Image(closest.get(0)).select('pr_wtr') ))
  tpw2 = ee.Image(ee.Algorithms.If(closest.size().eq(0), ee.Image.constant(-999.0),
                        ee.Algorithms.If(closest.size().eq(1), tpw1,
                        ee.Image(closest.get(1)).select('pr_wtr') )))

  time1 = ee.Number(ee.Algorithms.If(closest.size().eq(0), 1.0,
                        ee.Number(tpw1.get('DateDist')).divide(ee.Number(21600000)) ))
  time2 = ee.Number(ee.Algorithms.If(closest.size().lt(2), 0.0,
                        ee.Number(tpw2.get('DateDist')).divide(ee.Number(21600000)) ))

  tpw = tpw1.expression('tpw1*time2+tpw2*time1',
                            {'tpw1':tpw1,
                            'time1':time1,
                            'tpw2':tpw2,
                            'time2':time2
                            }).clip(image.geometry())

  # SMW coefficients are binned by TPW values
  # find the bin of each TPW value
  pos = tpw.expression(
    "value = (TPW>0 && TPW<=6) ? 0" + \
    ": (TPW>6 && TPW<=12) ? 1" + \
    ": (TPW>12 && TPW<=18) ? 2" + \
    ": (TPW>18 && TPW<=24) ? 3" + \
    ": (TPW>24 && TPW<=30) ? 4" + \
    ": (TPW>30 && TPW<=36) ? 5" + \
    ": (TPW>36 && TPW<=42) ? 6" + \
    ": (TPW>42 && TPW<=48) ? 7" + \
    ": (TPW>48 && TPW<=54) ? 8" + \
    ": (TPW>54) ? 9" + \
    ": 0",{'TPW': tpw}) \
    .clip(image.geometry())

  # add tpw to image as a band
  withTPW = (image.addBands(tpw.rename('TPW'),['TPW'])).addBands(pos.rename('TPWpos'),['TPWpos'])

  return withTPW




# get ASTER emissivity
aster = ee.Image("NASA/ASTER_GED/AG100_003")

#get ASTER FVC from NDVI
aster_ndvi = aster.select('ndvi').multiply(0.01)

aster_fvc = aster_ndvi.expression('((ndvi-ndvi_bg)/(ndvi_vg - ndvi_bg))**2',
  {'ndvi':aster_ndvi,'ndvi_bg':0.2,'ndvi_vg':0.86})
aster_fvc = aster_fvc.where(aster_fvc.lt(0.0),0.0)
aster_fvc = aster_fvc.where(aster_fvc.gt(1.0),1.0)

# bare ground emissivity functions for each band
def ASTERGEDemiss_bare_band10(image):
  return image.expression('(EM - 0.99*fvc)/(1.0-fvc)',{
    'EM':aster.select('emissivity_band10').multiply(0.001),
    'fvc':aster_fvc}) \
    .clip(image.geometry())


def ASTERGEDemiss_bare_band11(image):
  return image.expression('(EM - 0.99*fvc)/(1.0-fvc)',{
    'EM':aster.select('emissivity_band11').multiply(0.001),
    'fvc':aster_fvc}) \
    .clip(image.geometry())


def ASTERGEDemiss_bare_band12(image):
  return image.expression('(EM - 0.99*fvc)/(1.0-fvc)',{
    'EM':aster.select('emissivity_band12').multiply(0.001),
    'fvc':aster_fvc}) \
    .clip(image.geometry())


def ASTERGEDemiss_bare_band13(image):
  return image.expression('(EM - 0.99*fvc)/(1.0-fvc)',{
    'EM':aster.select('emissivity_band13').multiply(0.001),
    'fvc':aster_fvc}) \
    .clip(image.geometry())


def ASTERGEDemiss_bare_band14(image):
  return image.expression('(EM - 0.99*fvc)/(1.0-fvc)',{
    'EM':aster.select('emissivity_band14').multiply(0.001),
    'fvc':aster_fvc}) \
    .clip(image.geometry())


def EMaddBand(landsat, use_ndvi):
  def wrap(image):

    c13 = ee.Number(ee.Algorithms.If(landsat == 'L4',0.3222,
                            ee.Algorithms.If(landsat == 'L5',-0.0723,
                            ee.Algorithms.If(landsat == 'L7',0.2147,
                            0.6820))))
    c14 = ee.Number(ee.Algorithms.If(landsat == 'L4',0.6498,
                            ee.Algorithms.If(landsat == 'L5',1.0521,
                            ee.Algorithms.If(landsat == 'L7',0.7789,
                            0.2578))))
    c = ee.Number(ee.Algorithms.If(landsat == 'L4',0.0272,
                            ee.Algorithms.If(landsat == 'L5',0.0195,
                            ee.Algorithms.If(landsat == 'L7',0.0059,
                            0.0584))))

    # get ASTER emissivity
    # convolve to Landsat band
    emiss_bare = image.expression('c13*EM13 + c14*EM14 + c',{
      'EM13':ASTERGEDemiss_bare_band13(image),
      'EM14':ASTERGEDemiss_bare_band14(image),
      'c13':ee.Image(c13),
      'c14':ee.Image(c14),
      'c':ee.Image(c)
      })

    # compute the dynamic emissivity for Landsat
    EMd = image.expression('fvc*0.99+(1-fvc)*em_bare',
      {'fvc':image.select('FVC'),'em_bare':emiss_bare})

    # compute emissivity directly from ASTER
    # without vegetation correction
    # get ASTER emissivity
    aster = ee.Image("NASA/ASTER_GED/AG100_003") \
      .clip(image.geometry())
    EM0 = image.expression('c13*EM13 + c14*EM14 + c',{
      'EM13':aster.select('emissivity_band13').multiply(0.001),
      'EM14':aster.select('emissivity_band14').multiply(0.001),
      'c13':ee.Image(c13),
      'c14':ee.Image(c14),
      'c':ee.Image(c)
      })

    # select which emissivity to output based on user selection
    EM = ee.Image(ee.Algorithms.If(use_ndvi,EMd,EM0))

    return image.addBands(EM.rename('EM'))

  return wrap


def get_lookup_table(fc, prop_1, prop_2):
  reducer = ee.Reducer.toList().repeat(2)
  lookup = fc.reduceColumns(reducer, [prop_1, prop_2])
  return ee.List(lookup.get('list'))


def LSTaddBand(landsat):

  def wrap(image):

    # coefficients for the Statistical Mono-Window Algorithm
    coeff_SMW_L8 = ee.FeatureCollection([
    ee.Feature(None, {'TPWpos': 0, 'A': 0.9751, 'B': -205.8929, 'C': 212.7173}),
    ee.Feature(None, {'TPWpos': 1, 'A': 1.0090, 'B': -232.2750, 'C': 230.5698}),
    ee.Feature(None, {'TPWpos': 2, 'A': 1.0541, 'B': -253.1943, 'C': 238.9548}),
    ee.Feature(None, {'TPWpos': 3, 'A': 1.1282, 'B': -279.4212, 'C': 244.0772}),
    ee.Feature(None, {'TPWpos': 4, 'A': 1.1987, 'B': -307.4497, 'C': 251.8341}),
    ee.Feature(None, {'TPWpos': 5, 'A': 1.3205, 'B': -348.0228, 'C': 257.2740}),
    ee.Feature(None, {'TPWpos': 6, 'A': 1.4540, 'B': -393.1718, 'C': 263.5599}),
    ee.Feature(None, {'TPWpos': 7, 'A': 1.6350, 'B': -451.0790, 'C': 268.9405}),
    ee.Feature(None, {'TPWpos': 8, 'A': 1.5468, 'B': -429.5095, 'C': 275.0895}),
    ee.Feature(None, {'TPWpos': 9, 'A': 1.9403, 'B': -547.2681, 'C': 277.9953})
    ])

    # Select algorithm coefficients
    coeff_SMW = ee.FeatureCollection(coeff_SMW_L8)

    # Create lookups for the algorithm coefficients
    A_lookup = get_lookup_table(coeff_SMW, 'TPWpos', 'A')
    B_lookup = get_lookup_table(coeff_SMW, 'TPWpos', 'B')
    C_lookup = get_lookup_table(coeff_SMW, 'TPWpos', 'C')

    # Map coefficients to the image using the TPW bin position
    A_img = image.remap(A_lookup.get(0), A_lookup.get(1),0.0,'TPWpos').resample('bilinear')
    B_img = image.remap(B_lookup.get(0), B_lookup.get(1),0.0,'TPWpos').resample('bilinear')
    C_img = image.remap(C_lookup.get(0), C_lookup.get(1),0.0,'TPWpos').resample('bilinear')

    # select TIR band
    tir = ee.String(ee.Algorithms.If(landsat == 'L8','B10',
                        ee.Algorithms.If(landsat == 'L7','B6_VCID_1',
                        'B6')))
    # compute the LST
    lst = image.expression(
      'A*Tb1/em1 + B/em1 + C',
         {'A': A_img,
          'B': B_img,
          'C': C_img,
          'em1': image.select('EM'),
          'Tb1': image.select(tir)
         }).updateMask(image.select('TPW').lt(0).Not())


    return image.addBands(lst.rename('LST'))
  
  return wrap




# cloudmask for TOA data
def cloudmasktoa(image):
  qa = image.select('BQA')
  mask = qa.bitwiseAnd(1 << 4).eq(0)
  return image.updateMask(mask)


# cloudmask for SR data
def cloudmasksr(image):
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(1 << 3) \
    .Or(qa.bitwiseAnd(1 << 5))
  return image.updateMask(mask.Not())


def LSTcollection(landsat, date_start, date_end, geometry, image_limit, use_ndvi):

  # load TOA Radiance/Reflectance
  collection_dict = ee.Dictionary(COLLECTION.get(landsat))

  landsatTOA = ee.ImageCollection(collection_dict.get('TOA')) \
                .filter(ee.Filter.date(date_start, date_end)) \
                .filterBounds(geometry) \
                .map(cloudmasktoa)
                #.limit(image_limit,'CLOUD_COVER_LAND') \
    
  # load Surface Reflectance collection for NDVI
  landsatSR = ee.ImageCollection(collection_dict.get('SR')) \
                .filter(ee.Filter.date(date_start, date_end)) \
                .filterBounds(geometry) \
                .map(cloudmasksr) \
                .map(NDVIaddBand(landsat)) \
                .map(FVCaddBand(landsat)) \
                .map(NCEP_TPWaddBand) \
                .map(EMaddBand(landsat,use_ndvi))
                #.limit(image_limit,'CLOUD_COVER_LAND') \

# combine collections
# all channels from surface reflectance collection
# except tir channels: from TOA collection
# select TIR bands
  tir = ee.List(collection_dict.get('TIR'))
  landsatALL = (landsatSR.combine(landsatTOA.select(tir), True))

  # compute the LST
  landsatLST = landsatALL.map(LSTaddBand(landsat))

  return landsatLST




In [ ]:

# get landsat collection with added variables: NDVI, FVC, TPW, EM, LST
# link to the code that computes the Landsat LST

#LandsatColl = LandsatLST.collection(landsat, date_start, date_end, roi, use_ndvi)

LandsatColl = LSTcollection(landsat, date_start, date_end, roi, image_limit, use_ndvi).filter(ee.Filter.calendarRange(month_start, month_end, 'month'))

LSTmean = LandsatColl.select('LST').reduce(ee.Reducer.mean()).subtract(273.15)
#print(LSTmean)

# define "high LST" threshold
UrbanLSTmean = LSTmean.mask(WorldCover.eq(50))
UrbanAreaLSTReduction = UrbanLSTmean.reduceRegion(ee.Reducer.mean(),roi,30) # or ee.Reducer.percentile([50]) for median LST of region
thesholdAdder = 3 # degrees C above UrbanAreaReduction value at which to set threshold
TempThresValue = ee.Number(UrbanAreaLSTReduction.get('LST_mean')).multiply(100).round().divide(100).add(thesholdAdder).getInfo()
print(TempThresValue)

LSTmeanThres = LSTmean.updateMask(LSTmean.gte(TempThresValue))

#print(LSTmean.getInfo())
#print(LSTmean.projection().nominalScale().getInfo())

In [ ]:
#Add LST mean to map

cmap1 = ['blue', 'cyan', 'green', 'yellow', 'red']
Map.addLayer(LSTmean,{'min':20, 'max':45, 'palette':cmap1}, 'Mean land surface temperature C',False)
Map.addLayer(LSTmeanThres,{'min':20, 'max':45, 'palette':cmap1}, 'Mean land surface temperature C, areas '+str(thesholdAdder)+'C+ above built-up mean',True)

Map.addLayer(roi,{}, "Area of interest",True,0.3)
Map

In [ ]:
## calculations to determine LST by LULC class


## function to create image of means of toCount for each asClass

def getmeanbyclass(classvalue):
    return ee.Image(toCount.updateMask(asClass.eq(classvalue)) #.And(toCount.gt(0))) # uncomment And statement if you want include only pixels that meet both criteria
                    # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                    ).rename(ee.String('') #'class_count-'
                                       .cat(ee.Number(classvalue).toInt().format()))

## function to create image of count of each asClass
def getcountbyclass(classvalue):
    return ee.Image(toCount.updateMask(asClass.eq(classvalue)) #.And(toCount.gt(0))) # uncomment And statement if you want include only pixels that meet both criteria
                    # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                    ).rename(ee.String('class_count-') #'class_count-'
                                      .cat(ee.Number(classvalue).toInt().format()))

## function to create image of count of each asClass filtered by toCount
def getcountbyclassFilt(classvalue):
    return ee.Image(toCount.updateMask(asClass.eq(classvalue).And(toCount.gt(0))) # uncomment And statement if you want include only pixels that meet both criteria
                    # .unmask(0) # uncomment if you want to include all pixels not just pixels of classvalue
                    ).rename(ee.String('class_countFilt-') #'class_count-'
                                      .cat(ee.Number(classvalue).toInt().format()))

In [ ]:
## create image with each WorldCover class mean as a band

asClass = WorldCover
toCount = LSTmean 

meanbyclass=ee.Image(getmeanbyclass(10)).addBands([
  getmeanbyclass(20),  
  getmeanbyclass(30),  
  getmeanbyclass(40),  
  getmeanbyclass(50),  
  getmeanbyclass(60),
  getmeanbyclass(70),  
    getmeanbyclass(80), 
    getmeanbyclass(90), 
    getmeanbyclass(95), 
    getmeanbyclass(100), 
])

## create image with each WorldCover class count as a band

countbyclass=ee.Image(getcountbyclass(10)).addBands([
  getcountbyclass(20),  
  getcountbyclass(30),  
  getcountbyclass(40),  
  getcountbyclass(50),  
  getcountbyclass(60),
  getcountbyclass(70),  
    getcountbyclass(80), 
    getcountbyclass(90), 
    getcountbyclass(95), 
    getcountbyclass(100), 
])

## create image with each WorldCover class count above LST threshold as a band
toCount = LSTmeanThres 

countbyclassFilt=ee.Image(getcountbyclassFilt(10)).addBands([
  getcountbyclassFilt(20),  
  getcountbyclassFilt(30),  
  getcountbyclassFilt(40),  
  getcountbyclassFilt(50),  
  getcountbyclassFilt(60),
  getcountbyclassFilt(70),  
    getcountbyclassFilt(80), 
    getcountbyclassFilt(90), 
    getcountbyclassFilt(95), 
    getcountbyclassFilt(100), 
])

#print('meanbyclass', meanbyclass.getInfo())
#print('countbyclass', countbyclass.getInfo())
#print('countbyclassFilt', countbyclassFilt.getInfo())

#Map.addLayer(meanbyclass.select('50'),{},"meanbyWCclass")
#Map.addLayer(countbyclassFilt.select('class_countFilt-10'),{},"countbyFiltWCclass")
#Map.addLayer(countbyclass.select('class_count-10'),{},"countbyWCclass")

In [ ]:
## create image with each ULU class mean as a band

asClass = ULUmaskedESA
toCount = LSTmean 

meanbyclassULU=ee.Image(getmeanbyclass(0)).addBands([
  getmeanbyclass(1),  
  getmeanbyclass(2),  
  getmeanbyclass(3),  
  getmeanbyclass(4),  
  getmeanbyclass(5),
  getmeanbyclass(6) 
])

## create image with each ULU class count as a band

countbyclassULU=ee.Image(getcountbyclass(0)).addBands([
  getcountbyclass(1),  
  getcountbyclass(2),  
  getcountbyclass(3),  
  getcountbyclass(4),  
  getcountbyclass(5),
  getcountbyclass(6)
])

## create image with each WorldCover class count above LST threshold as a band
toCount = LSTmeanThres 

countbyclassFiltULU=ee.Image(getcountbyclassFilt(0)).addBands([
  getcountbyclassFilt(1),  
  getcountbyclassFilt(2),  
  getcountbyclassFilt(3),  
  getcountbyclassFilt(4),  
  getcountbyclassFilt(5),
  getcountbyclassFilt(6)
])

#Map.addLayer(meanbyclassULU.select('1'),{},"meanbyULUclass")
#Map.addLayer(countbyclassFiltULU.select('class_countFilt-0'),{},"countbyFiltULUclass")
#Map.addLayer(countbyclassULU.select('class_count-0'),{},"countbyULUclass")

In [ ]:
## create FeatureCollection with mean of count for each class for each feature

histo=meanbyclass.reduceRegions(
  reducer= ee.Reducer.mean(), 
  collection= Districts, 
  scale= 10, 
  tileScale= 1
)

histo=countbyclass.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= histo, 
  scale= 10, 
  tileScale= 1
)

histo=countbyclassFilt.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= histo, 
  scale= 10, 
  tileScale= 1
)

histo=meanbyclassULU.reduceRegions(
  reducer= ee.Reducer.mean(), 
  collection= histo, 
  scale= 10, 
  tileScale= 1
)

histo=countbyclassULU.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= histo, 
  scale= 10, 
  tileScale= 1
)

histo=countbyclassFiltULU.reduceRegions(
  reducer= ee.Reducer.count(), 
  collection= histo, 
  scale= 10, 
  tileScale= 1
)

#print('histo:', histo.limit(1,'class_count-10',False).getInfo())
print('histo:', histo.first().toDictionary().getInfo())

In [ ]:
## Define function to normalize count as percent of all pixels in each feature and create new properties with the values

def count_to_percent(feat):
    feat=ee.Feature(feat)
    hist=ee.Dictionary(feat.toDictionary(['10','20','30','40','50','60','70','80','90','95','100']))
    hist=hist.set('10',hist.get('10',0))
    hist=hist.set('20',hist.get('20',0))
    hist=hist.set('30',hist.get('30',0))
    hist=hist.set('40',hist.get('40',0))
    hist=hist.set('50',hist.get('50',0))
    hist=hist.set('60',hist.get('60',0))
    hist=hist.set('70',hist.get('70',0))
    hist=hist.set('80',hist.get('80',0))
    hist=hist.set('90',hist.get('90',0))
    hist=hist.set('95',hist.get('95',0))
    hist=hist.set('100',hist.get('100',0))
    
    def pct_hist(k,v):
        # convert whole number (0-100) to decimal percent (0-1)
        return ee.Number(v)
    
    meansLULC = hist.map(pct_hist)
    
    histC=ee.Dictionary(feat.toDictionary(['class_count-10','class_count-20','class_count-30','class_count-40','class_count-50','class_count-60','class_count-70','class_count-80','class_count-90','class_count-95','class_count-100']))
    histC=histC.set('10',histC.get('class_count-10',0))
    histC=histC.set('20',histC.get('class_count-20',0))
    histC=histC.set('30',histC.get('class_count-30',0))
    histC=histC.set('40',histC.get('class_count-40',0))
    histC=histC.set('50',histC.get('class_count-50',0))
    histC=histC.set('60',histC.get('class_count-60',0))
    histC=histC.set('70',histC.get('class_count-70',0))
    histC=histC.set('80',histC.get('class_count-80',0))
    histC=histC.set('90',histC.get('class_count-90',0))
    histC=histC.set('95',histC.get('class_count-95',0))
    histC=histC.set('100',histC.get('class_count-100',0))
    
    histCfilt=ee.Dictionary(feat.toDictionary(['class_countFilt-10','class_countFilt-20','class_countFilt-30','class_countFilt-40','class_countFilt-50','class_countFilt-60','class_countFilt-70','class_countFilt-80','class_countFilt-90','class_countFilt-95','class_countFilt-100']))
    histCfilt=histCfilt.set('10',histCfilt.get('class_countFilt-10',0))
    histCfilt=histCfilt.set('20',histCfilt.get('class_countFilt-20',0))
    histCfilt=histCfilt.set('30',histCfilt.get('class_countFilt-30',0))
    histCfilt=histCfilt.set('40',histCfilt.get('class_countFilt-40',0))
    histCfilt=histCfilt.set('50',histCfilt.get('class_countFilt-50',0))
    histCfilt=histCfilt.set('60',histCfilt.get('class_countFilt-60',0))
    histCfilt=histCfilt.set('70',histCfilt.get('class_countFilt-70',0))
    histCfilt=histCfilt.set('80',histCfilt.get('class_countFilt-80',0))
    histCfilt=histCfilt.set('90',histCfilt.get('class_countFilt-90',0))
    histCfilt=histCfilt.set('95',histCfilt.get('class_countFilt-95',0))
    histCfilt=histCfilt.set('100',histCfilt.get('class_countFilt-100',0))
    
    def area_hist(k,v):
        # convert 10m pixel count of class to KM2 of class
        return ee.Number(v).multiply(ee.Number(100)).multiply(ee.Number(0.000001))
    
    classAreas = histC.map(area_hist)
    classFiltAreas = histCfilt.map(area_hist)

    
    FeatArea = feat.area(0.001).multiply(0.000001)
    cityID = Areaofinterest
    geo_level = feat.getString("geo_level")
    geo_name = feat.getString("geo_name").split(' ').join('_')
    #geo_name = feat.getString("Sub_City").cat(ee.String("-")).cat(feat.getString("Woreda"))
    #geo_name = feat.getString("city_name_viz").split(' ').join('_')
    #geo_name = feat.getString("City Name")
    geo_id = ee.String(cityID+"-").cat(geo_name)
    source = "Landsat, ESA WorldCover, WRI ULU"

    totalPixels=hist.values()
    
    histULU=ee.Dictionary(feat.toDictionary(['0','1','2','3','4','5','6']))
    histULU=histULU.set('0',histULU.get('0',0))
    histULU=histULU.set('1',histULU.get('1',0))
    histULU=histULU.set('2',histULU.get('2',0))
    histULU=histULU.set('3',histULU.get('3',0))
    histULU=histULU.set('4',histULU.get('4',0))
    histULU=histULU.set('5',histULU.get('5',0))
    histULU=histULU.set('6',histULU.get('6',0))
    
    meansULU = histULU.map(pct_hist)
        
    histULUc=ee.Dictionary(feat.toDictionary(['class_count-0','class_count-1','class_count-2','class_count-3','class_count-4','class_count-5','class_count-6']))
    histULUc=histULUc.set('0',histULUc.get('class_count-0',0))
    histULUc=histULUc.set('1',histULUc.get('class_count-1',0))
    histULUc=histULUc.set('2',histULUc.get('class_count-2',0))
    histULUc=histULUc.set('3',histULUc.get('class_count-3',0))
    histULUc=histULUc.set('4',histULUc.get('class_count-4',0))
    histULUc=histULUc.set('5',histULUc.get('class_count-5',0))
    histULUc=histULUc.set('6',histULUc.get('class_count-6',0))
    
    histULUcFilt=ee.Dictionary(feat.toDictionary(['class_countFilt-0','class_countFilt-1','class_countFilt-2','class_countFilt-3','class_countFilt-4','class_countFilt-5','class_countFilt-6']))
    histULUcFilt=histULUcFilt.set('0',histULUcFilt.get('class_countFilt-0',0))
    histULUcFilt=histULUcFilt.set('1',histULUcFilt.get('class_countFilt-1',0))
    histULUcFilt=histULUcFilt.set('2',histULUcFilt.get('class_countFilt-2',0))
    histULUcFilt=histULUcFilt.set('3',histULUcFilt.get('class_countFilt-3',0))
    histULUcFilt=histULUcFilt.set('4',histULUcFilt.get('class_countFilt-4',0))
    histULUcFilt=histULUcFilt.set('5',histULUcFilt.get('class_countFilt-5',0))
    histULUcFilt=histULUcFilt.set('6',histULUcFilt.get('class_countFilt-6',0))

    classAreasULU = histULUc.map(area_hist)
    classFiltAreasULU = histULUcFilt.map(area_hist)

    return feat.set({
        'LC10LST': meansLULC.getNumber('10'),
        'LC20LST': meansLULC.getNumber('20'),
        'LC30LST': meansLULC.getNumber('30'),
        'LC40LST': meansLULC.getNumber('40'),
        'LC50LST': meansLULC.getNumber('50'),
        'LC60LST': meansLULC.getNumber('60'),
        'LC70LST': meansLULC.getNumber('70'),
        'LC80LST': meansLULC.getNumber('80'),
        'LC90LST': meansLULC.getNumber('90'),
        'LC95LST': meansLULC.getNumber('95'),
        'LC100LST': meansLULC.getNumber('100'),
        'TotalareaKM2': FeatArea,
        'TotalPixels': totalPixels,
        'geo_level': geo_level,
        'geo_name': geo_name,
        'geo_id': geo_id,
        'date_start': date_start,
        'date_end': date_end,
        'source':source,
        'LC10areaKM2': classAreas.getNumber('10'),
        'LC20areaKM2': classAreas.getNumber('20'),
        'LC30areaKM2': classAreas.getNumber('30'),
        'LC40areaKM2': classAreas.getNumber('40'),
        'LC50areaKM2': classAreas.getNumber('50'),
        'LC60areaKM2': classAreas.getNumber('60'),
        'LC70areaKM2': classAreas.getNumber('70'),
        'LC80areaKM2': classAreas.getNumber('80'),
        'LC90areaKM2': classAreas.getNumber('90'),
        'LC95areaKM2': classAreas.getNumber('95'),
        'LC100areaKM2': classAreas.getNumber('100'),
        'LC10highLSTpct': classFiltAreas.getNumber('10').divide(classAreas.getNumber('10')),
        'LC20highLSTpct': classFiltAreas.getNumber('20').divide(classAreas.getNumber('20')),
        'LC30highLSTpct': classFiltAreas.getNumber('30').divide(classAreas.getNumber('30')),
        'LC40highLSTpct': classFiltAreas.getNumber('40').divide(classAreas.getNumber('40')),
        'LC50highLSTpct': classFiltAreas.getNumber('50').divide(classAreas.getNumber('50')),
        'LC60highLSTpct': classFiltAreas.getNumber('60').divide(classAreas.getNumber('60')),
        'LC70highLSTpct': classFiltAreas.getNumber('70').divide(classAreas.getNumber('70')),
        'LC80highLSTpct': classFiltAreas.getNumber('80').divide(classAreas.getNumber('80')),
        'LC90highLSTpct': classFiltAreas.getNumber('90').divide(classAreas.getNumber('90')),
        'LC95highLSTpct': classFiltAreas.getNumber('95').divide(classAreas.getNumber('95')),
        'LC100highLSTpct': classFiltAreas.getNumber('100').divide(classAreas.getNumber('100')),
        'ULU0LST': meansULU.getNumber('0'),
        'ULU1LST': meansULU.getNumber('1'),
        'ULU2LST': meansULU.getNumber('2'),
        'ULU3LST': meansULU.getNumber('3'),
        'ULU4LST': meansULU.getNumber('4'),
        'ULU5LST': meansULU.getNumber('5'),
        'ULU6LST': meansULU.getNumber('6'),
        'ULU0areaKM2': classAreasULU.getNumber('0'),
        'ULU1areaKM2': classAreasULU.getNumber('1'),
        'ULU2areaKM2': classAreasULU.getNumber('2'),
        'ULU3areaKM2': classAreasULU.getNumber('3'),
        'ULU4areaKM2': classAreasULU.getNumber('4'),
        'ULU5areaKM2': classAreasULU.getNumber('5'),
        'ULU6areaKM2': classAreasULU.getNumber('6'),
        'ULU0highLSTpct': classFiltAreasULU.getNumber('0').divide(classAreasULU.getNumber('0')),
        'ULU1highLSTpct': classFiltAreasULU.getNumber('1').divide(classAreasULU.getNumber('1')),
        'ULU2highLSTpct': classFiltAreasULU.getNumber('2').divide(classAreasULU.getNumber('2')),
        'ULU3highLSTpct': classFiltAreasULU.getNumber('3').divide(classAreasULU.getNumber('3')),
        'ULU4highLSTpct': classFiltAreasULU.getNumber('4').divide(classAreasULU.getNumber('4')),
        'ULU5highLSTpct': classFiltAreasULU.getNumber('5').divide(classAreasULU.getNumber('5')),
        'ULU6highLSTpct': classFiltAreasULU.getNumber('6').divide(classAreasULU.getNumber('6')),
    })

In [ ]:
## update FeatureCollection with percents

lst_means=histo.map(count_to_percent)

#print('LST stats by Land Cover class for Districts',lst_means.limit(1).getInfo());
print('LST stats by Land Cover class for Districts',lst_means.first().toDictionary().getInfo())

In [ ]:
## render on map percent tree cover by class from feature collection

empty = ee.Image().byte()
Tpctfills = empty.paint(**{'featureCollection': lst_means,'color': 'LC50highLSTpct'})

fillspalette = ['green', 'red']
cmap1 = ['blue', 'cyan', 'green', 'yellow', 'red']
Map.addLayer(Tpctfills, {'palette': fillspalette,'min':0,'max':1}, '% of built areas with high mean land surface temperature', True, 0.65)
Map

In [ ]:
## select properties to keep, sort features and create data frame to display properties
lst_meansSort = lst_means.select([
    'TotalareaKM2',
    'geo_level',
    'geo_name',
    'geo_id',
    'date_start',
    'date_end',
    'source',
    'LC10LST','LC20LST','LC30LST','LC40LST','LC50LST','LC60LST','LC70LST','LC80LST','LC90LST','LC95LST','LC100LST',
    'ULU0LST','ULU1LST','ULU2LST','ULU3LST','ULU4LST','ULU5LST','ULU6LST',
    'LC10highLSTpct','LC20highLSTpct','LC30highLSTpct','LC40highLSTpct','LC50highLSTpct','LC60highLSTpct','LC70highLSTpct','LC80highLSTpct','LC90highLSTpct','LC95highLSTpct','LC100highLSTpct',
    'ULU0highLSTpct','ULU1highLSTpct','ULU2highLSTpct','ULU3highLSTpct','ULU4highLSTpct','ULU5highLSTpct','ULU6highLSTpct',
]).sort('LC50highLSTpct', False) #
#print('Tree cover sorted version',tree_pctsSort.limit(1).getInfo());

In [ ]:
df = geemap.ee_to_pandas(lst_meansSort)
df

In [ ]:
## display features in chart

import geemap.chart as chart

xProperty = 'geo_name' #,"Woreda"
yProperties = ['LC50highLSTpct'] # ,'LC50areaKM2'

options = {
    'xlabel': "District",
    'ylabel': "Percent of built area with LST of "+str(thesholdAdder)+"C+ above built-up mean during heat wave",
    "legend_location": "top-right",
    "height": "500px",
}

chart.feature_byFeature(lst_meansSort, xProperty, yProperties, **options)